In [2]:
# Load fine-tuned DenseNet model
from tensorflow.keras.models import load_model, Model
model = load_model(r"F:\Multi_Chronic_Disease_Detection\model\densenet_new_finetuned_v3.h5")
feature_extractor = Model(inputs=model.input, outputs=model.layers[-3].output)  # Layer before Dropout


In [5]:
# Load the trained model
from tensorflow.keras.models import load_model

cnn_model = load_model(r"F:\Multi_Chronic_Disease_Detection\model\densenet_new_finetuned_v3.h5")

# Train
X_train, y_train, class_indices = extract_features_batch(cnn_model, r"F:\Multi_Chronic_Disease_Detection\dataset\train")

# Validation
X_val, y_val, _ = extract_features_batch(cnn_model, r"F:\Multi_Chronic_Disease_Detection\dataset\validation")

# Test
X_test, y_test, _ = extract_features_batch(cnn_model, r"F:\Multi_Chronic_Disease_Detection\dataset\test")


Found 22000 images belonging to 5 classes.
688/688 [==============================] - 496s 715ms/step
Found 2750 images belonging to 5 classes.
86/86 [==============================] - 83s 945ms/step
Found 2750 images belonging to 5 classes.
86/86 [==============================] - 82s 932ms/step


In [6]:
import numpy as np

# Save train features
np.save("X_train.npy", X_train)
np.save("y_train.npy", y_train)

# Save validation features
np.save("X_val.npy", X_val)
np.save("y_val.npy", y_val)

# Save test features
np.save("X_test.npy", X_test)
np.save("y_test.npy", y_test)

# Optional: Save class label mapping
import json
with open("class_indices.json", "w") as f:
    json.dump(class_indices, f)


In [7]:
X_train = np.load("X_train.npy")
y_train = np.load("y_train.npy")

X_val = np.load("X_val.npy")
y_val = np.load("y_val.npy")

X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")

import json
with open("class_indices.json", "r") as f:
    class_indices = json.load(f)


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on extracted CNN features
rf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [5]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import joblib

# Load data
X_train = np.load("F:/Multi_Chronic_Disease_Detection/notebooks/X_train.npy")
X_val = np.load("F:/Multi_Chronic_Disease_Detection/notebooks/X_val.npy")
X_test = np.load("F:/Multi_Chronic_Disease_Detection/notebooks/X_test.npy")
y_train = np.load("F:/Multi_Chronic_Disease_Detection/notebooks/y_train.npy")
y_val = np.load("F:/Multi_Chronic_Disease_Detection/notebooks/y_val.npy")
y_test = np.load("F:/Multi_Chronic_Disease_Detection/notebooks/y_test.npy")

# Combine train + val
X_train_combined = np.concatenate((X_train, X_val), axis=0)
y_train_combined = np.concatenate((y_train, y_val), axis=0)

# Fast base models
rf = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
xgb = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, subsample=0.8,
                    colsample_bytree=0.8, random_state=42, n_jobs=-1,
                    use_label_encoder=False, eval_metric='mlogloss')

# Meta model (smaller)
meta_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=42)

# Train base models separately
rf.fit(X_train_combined, y_train_combined)
xgb.fit(X_train_combined, y_train_combined)

# Create stacking with prefit base models
stacking_model = StackingClassifier(
    estimators=[('rf', rf), ('xgb', xgb)],
    final_estimator=meta_model,
    cv='prefit',  # No internal CV
    n_jobs=-1,
    passthrough=True
)

# Train stacking model
stacking_model.fit(X_train_combined, y_train_combined)

# Evaluate
y_pred = stacking_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(" Fast Stacking Ensemble Accuracy:", accuracy)
print("\n Classification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)

# Save model
joblib.dump(stacking_model, "F:/Multi_Chronic_Disease_Detection/notebooks/fast_rf_xgb_stack2.pkl")
print("\nFast stacking model saved as fast_rf_xgb_stack2.pkl")


F:\anaconda\envs\tf_env\lib\site-packages\xgboost\training.py:183: UserWarning: [16:34:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


 Fast Stacking Ensemble Accuracy: 0.9225454545454546

 Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.88      0.86       550
           1       0.87      0.76      0.81       550
           2       0.90      0.97      0.93       550
           3       1.00      1.00      1.00       550
           4       1.00      1.00      1.00       550

    accuracy                           0.92      2750
   macro avg       0.92      0.92      0.92      2750
weighted avg       0.92      0.92      0.92      2750


Confusion Matrix:
 [[486  53  11   0   0]
 [ 83 418  48   0   1]
 [  6   7 536   0   1]
 [  1   0   0 549   0]
 [  0   0   2   0 548]]

Fast stacking model saved as fast_rf_xgb_stack2.pkl
